In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [ ]:
# הורדת משאבים ל-NLP
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
# טען את הדאטה
df = pd.read_csv("data_with_buzz.csv")
print(f"טעינת דאטה: {df.shape[0]} שורות, {df.shape[1]} עמודות")

טעינת דאטה: 23400 שורות, 19 עמודות


In [ ]:
######## טיפול בערכים חסרים ##########
######################################



#טיפול בערכים חסרים לטקסט
df['Title'] = df['Title'].fillna('')
df['Selftext'] = df['Selftext'].fillna('')

# המרת עמודות תאריך
df['Account Age Date'] = pd.to_datetime(df['Account Age'], errors='coerce', dayfirst=True)
df['Created At'] = pd.to_datetime(df['Created At'], errors='coerce', dayfirst=True)

# יצירת עמודת גיל חשבון בשנים (נכון)
df['Account Age (Years)'] = (df['Created At'] - df['Account Age Date']).dt.days / 365.25

# מילוי ערכים חסרים/שליליים בחציון
account_age_median = df['Account Age (Years)'].median()
df['Account Age (Years)'] = df['Account Age (Years)'].fillna(account_age_median)
df.loc[df['Account Age (Years)'] < 0, 'Account Age (Years)'] = account_age_median

# Karma
df['Author Link Karma'] = pd.to_numeric(df['Author Link Karma'], errors='coerce')
df['Author Link Karma'] = df['Author Link Karma'].fillna(df['Author Link Karma'].median())



In [ ]:
#איחוד לטקסט מלא
df['FullText'] = (df['Title'] + ' ' + df['Selftext']).str.strip()
#המרה לקטן שיהיה אחיד
df['FullText'] = df['FullText'].str.lower()

In [ ]:
#ניקוי טקסט
def clean_text(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'@\w+|#\w+', '', text)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

df['FullText'] = df['FullText'].apply(clean_text)

stop_words = set(stopwords.words('english'))
df['FullText'] = df['FullText'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))


In [ ]:
lemmatizer = WordNetLemmatizer()
def lemmatize_text(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
df['FullText'] = df['FullText'].apply(lemmatize_text)


In [ ]:
# חישוב עמודת אורך טקסט
df['FullText Length'] = df['FullText'].str.len()

In [ ]:
# המרת Created At לתאריך ופיצ'רים נוספים
df['Year'] = df['Created At'].dt.year
df['Month'] = df['Created At'].dt.month
df['DayOfWeek'] = df['Created At'].dt.dayofweek

In [ ]:
# הסרת כפיליות
df = df.drop_duplicates()

In [ ]:
# מיפוי מחדש לערכים מספריים
sentiment_map = {
    'Neutral': 0,
    'Positive': 1,
    'Negative': -1
}
df['Sentiment'] = df['Sentiment'].map(sentiment_map)

In [ ]:
###########  שמירה   #########
##############################


# רשימת עמודות למחיקה
columns_to_drop = [
    'Title',            # עמודת טקסט ראשי
    'Selftext',         # עמודת טקסט משני
    'Account Age',      # תאריך מקורי, אחרי שהומר למספר
    'Account Age Date', # עמודת עזר
    'Upvotes', 'Downvotes', # אם לא דרושות לך
    'Title Length', 'Post Length' #אורך פוסט יש חיבור
]

# הסר עמודות שלא רלוונטיות (רק אם הן קיימות ב-DataFrame)
for col in columns_to_drop:
    if col in df.columns:
        df = df.drop(columns=[col])

# קביעת סדר העמודות הסופי
columns_order = (
    ['FullText', 'FullText Length'] +                # קודם FullText ואז FullText Length
    [col for col in df.columns if col not in ['FullText', 'FullText Length', 'buzz']] +
    ['buzz']                                         # buzz אחרונה
)

df = df[columns_order]

# עכשיו שמירה של כל מה שנשאר
df.to_csv("data_final.csv", index=False)
print("data_final.csv")


data_final.csv


In [ ]:
print(f"טעינת דאטה: {df.shape[0]} שורות, {df.shape[1]} עמודות")

טעינת דאטה: 23338 שורות, 20 עמודות
